## Formation of the Greedy Algorithm

## Generating Points and Distances from TSP file

In [ ]:
import matplotlib.pyplot as plt
import math
import random

n = 50
random.seed(13)

def x_of(p):
    (x, _) = p
    return x

def y_of(p):
    (_, y) = p
    return y

def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1 - x2)**2 + (y1 - y2)**2) * 700)

## Replacing  the generation of points  from the TSP with the generation of job processing times

In [ ]:
N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC


## The Greedy Algorithm for TSP

In [ ]:
def nearest(p, x):
    idx = 0
    mindist = dist(p[0], x)
    for i in range(1, len(p)):
        d = dist(p[i], x)
        if d < mindist:
            mindist = d
            idx = i
    return p[idx]

def greedy(p):
    p = p.copy()
    x = p[0]
    path = [x]
    p.remove(x)
    while len(p) > 0:
        x = nearest(p, x)
        path.append(x)
        p.remove(x)
    path.append(path[0])
    return path


In [ ]:
geeksforgeeks

def printJobScheduling(arr, t):
 
    # length of array
    n = len(arr)
 
    # Sort all jobs according to
    # decreasing order of profit
    for i in range(n):
        for j in range(n - 1 - i):
            if arr[j][2] < arr[j + 1][2]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
 
    # To keep track of free time slots
    result = [False] * t
 
    # To store result (Sequence of jobs)
    job = ['-1'] * t
 
    # Iterate through all given jobs
    for i in range(len(arr)):
 
        # Find a free slot for this job
        # (Note that we start from the
        # last possible slot)
        for j in range(min(t - 1, arr[i][1] - 1), -1, -1):
 
            # Free slot found
            if result[j] is False:
                result[j] = True
                job[j] = arr[i][0]
                break
 
    # print the sequence
    print(job)
 

## Adapting the above Approach to the  job scheduling by selecting the job that can start the earliest based on the current machine availability

In [ ]:
def greedy_schedule(PROC, N_JOBS, N_MACHINES):
    start = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stop = [[0] * N_MACHINES for _ in range(N_JOBS)]
    idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    machine_available = [0] * N_MACHINES
    last_job_completed = [-1] * N_MACHINES
    job_sequence = []

    remaining_jobs = list(range(N_JOBS))

    while remaining_jobs:
        next_job = min(remaining_jobs, key=lambda j: min(machine_available[m] + PROC[j][m] for m in range(N_MACHINES)))
        job_sequence.append(next_job)
        remaining_jobs.remove(next_job)

        for machine in range(N_MACHINES):
            start_time = max(machine_available[machine], stop[next_job][machine - 1] if machine > 0 else 0)
            start[next_job][machine] = start_time
            idle[next_job][machine] = start_time - (stop[last_job_completed[machine]][machine] if last_job_completed[machine] != -1 else 0)
            stop[next_job][machine] = start_time + PROC[next_job][machine]
            machine_available[machine] = stop[next_job][machine]

            if last_job_completed[machine] != -1:
                wait[next_job][machine] = start_time - stop[last_job_completed[machine]][machine]

            last_job_completed[machine] = next_job

    objective_value = max(machine_available)
    return start, stop, idle, wait, job_sequence, objective_value


In [ ]:
def print_greedy_schedule(PROC, N_JOBS, N_MACHINES):
    start_time = time.time()
    start, stop, idle, wait, job_sequence, objective_value = greedy_schedule(PROC, N_JOBS, N_MACHINES)
    end_time = time.time()
    runtime = end_time - start_time
    
    print("Job Sequence:", job_sequence)
    print("Objective Value:", objective_value)
    print("Runtime: {:.4f} seconds".format(runtime))

    #from the JobScheduling Solution in Jupyter Notebook

    row = "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    row += "|         |"
    for m in range(N_MACHINES):
        row += f"            | Machine: {m} |"
    row += "\n"

    for j in job_sequence:
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Idle: {idle[j][m]:4d} |"
        row += "\n"
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{start[j][m]:4d} |"
        row += "\n"
        row += f"| Job: {j:2d} |"
        for m in range(N_MACHINES):
            row += f" Wait: {wait[j][m]:4d} | Proc: {PROC[j][m]:4d} |"
        row += "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Stop: {stop[j][m]:4d} |"
        row += "\n"
    row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    print(row)


In [ ]:
job_counts = [5, 6, 7, 8, 9, 10]

print("Greedy Algorithm:")
for N_JOBS in job_counts:
    PROC = generateData(N_JOBS=N_JOBS)
    print(f"\nComparing for N_JOBS = {N_JOBS} with Greedy Algorithm:")
    print_greedy_schedule(PROC, N_JOBS, N_MACHINES)
    print("\n")


## Formation of the Genetic Algorithm

In [ ]:
import random
from deap import base, creator, tools, algorithms

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

# Example usage of data generation
PROC = generateData()
print("Processing Times:", PROC)


In [ ]:
import random
from deap import base, creator, tools, algorithms

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

# Define the fitness function
def calculate_makespan(individual, PROC, N_MACHINES):
    machine_time = [0] * N_MACHINES
    job_end_times = [0] * len(individual)
    for job in individual:
        for machine in range(N_MACHINES):
            start_time = max(machine_time[machine], job_end_times[job])
            duration = PROC[job][machine]
            machine_time[machine] = start_time + duration
            job_end_times[job] = machine_time[machine]
    return max(machine_time),

# Example usage of data generation and fitness function
PROC = generateData()
individual = list(range(len(PROC)))
random.shuffle(individual)
print("Makespan:", calculate_makespan(individual, PROC, N_MACHINES))


In [ ]:
From Githum.com/DEAP
toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

In [ ]:
import random

from deap import base
from deap import creator
from deap import tools

IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

In [ ]:
import random
from deap import base, creator, tools, algorithms

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

def calculate_makespan(individual, PROC, N_MACHINES):
    machine_time = [0] * N_MACHINES
    job_end_times = [0] * len(individual)
    for job in individual:
        for machine in range(N_MACHINES):
            start_time = max(machine_time[machine], job_end_times[job])
            duration = PROC[job][machine]
            machine_time[machine] = start_time + duration
            job_end_times[job] = machine_time[machine]
    return max(machine_time),

# DEAP toolbox setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(len(PROC)), len(PROC))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=50)
toolbox.register("evaluate", calculate_makespan, PROC=PROC, N_MACHINES=N_MACHINES)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
import random
from deap import base, creator, tools, algorithms

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

def calculate_makespan(individual, PROC, N_MACHINES):
    machine_time = [0] * N_MACHINES
    job_end_times = [0] * len(individual)
    for job in individual:
        for machine in range(N_MACHINES):
            start_time = max(machine_time[machine], job_end_times[job])
            duration = PROC[job][machine]
            machine_time[machine] = start_time + duration
            job_end_times[job] = machine_time[machine]
    return max(machine_time),

# DEAP toolbox setup
if 'FitnessMin' not in creator.__dict__:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
if 'Individual' not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(5), 5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=50)
toolbox.register("evaluate", calculate_makespan, PROC=generateData(), N_MACHINES=N_MACHINES)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Running the Genetic Algorithm
population = toolbox.population()
ngen = 100
cxpb = 0.7
mutpb = 0.2

algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=False)
best_individual = tools.selBest(population, 1)[0]
best_fitness = calculate_makespan(best_individual, generateData(), N_MACHINES)[0]
print("Best Individual:", best_individual)
print("Best Fitness (Makespan):", best_fitness)


In [ ]:
# Final Version: Added Runtime Measurement and Printing Schedule
import random
import time
from deap import base, creator, tools, algorithms

N_MACHINES = 4

def generateData(seed=5640, N_JOBS=5):
    random.seed(seed)
    PROC = [[random.randint(1, 9) for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    return PROC

def calculate_makespan(individual, PROC, N_MACHINES):
    machine_time = [0] * N_MACHINES
    job_end_times = [0] * len(individual)
    for job in individual:
        for machine in range(N_MACHINES):
            start_time = max(machine_time[machine], job_end_times[job])
            duration = PROC[job][machine]
            machine_time[machine] = start_time + duration
            job_end_times[job] = machine_time[machine]
    return max(machine_time),

# DEAP toolbox setup
if 'FitnessMin' not in creator.__dict__:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
if 'Individual' not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(5), 5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=50)
toolbox.register("evaluate", calculate_makespan, PROC=generateData(), N_MACHINES=N_MACHINES)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

def genetic_algorithm_schedule(PROC, N_JOBS, N_MACHINES):
    population = toolbox.population()
    ngen = 100
    cxpb = 0.7
    mutpb = 0.2

    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=False)
    best_individual = tools.selBest(population, 1)[0]
    best_fitness = calculate_makespan(best_individual, PROC, N_MACHINES)[0]
    
    return best_individual, best_fitness

def print_genetic_schedule(PROC, N_JOBS, N_MACHINES):
    start_time = time.time()
    best_individual, best_fitness = genetic_algorithm_schedule(PROC, N_JOBS, N_MACHINES)
    end_time = time.time()
    runtime = end_time - start_time
    
    print("Job Sequence:", best_individual)
    print("Objective Value:", best_fitness)
    print("Runtime: {:.4f} seconds".format(runtime))

    machine_time = [0] * N_MACHINES
    job_end_times = [0] * N_JOBS
    start = [[] for _ in range(N_JOBS)]
    stop = [[] for _ in range(N_JOBS)]
    idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait = [[0] * N_MACHINES for _ in range(N_JOBS)]
    
    for job in best_individual:
        for machine in range(N_MACHINES):
            start_time = max(machine_time[machine], job_end_times[job])
            start[job].append(start_time)
            stop_time = start_time + PROC[job][machine]
            stop[job].append(stop_time)
            machine_time[machine] = stop_time
            job_end_times[job] = stop_time

    row = "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    row += "|         |"
    for m in range(N_MACHINES):
        row += f"            | Machine: {m} |"
    row += "\n"
    for j in best_individual:
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            idle_time = start[j][m] - (stop[j][m-1] if m > 0 else 0)
            row += f"       {' ':4s} | Idle: {max(0, idle_time):4d} |"
        row += "\n"
        row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
        row += "|         |"
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | Start:{start[j][m]:4d} |"
        row += "\n"
        row += f"| Job: {j:2d} |"
        for m in range(N_MACHINES):
            row += f" Wait: {wait[j][m]:4d} | Proc: {PROC[j][m]:4d} | Stop: {stop[j][m]:4d} |"
        row += "\n"
    row += "|---------|" + "------------|------------|" * N_MACHINES + "\n"
    print(row)

# The Genetic Algorithm running for different N_JOBS
job_counts = [5, 6, 7, 8, 9, 10]

print("Genetic Algorithm:")
for N_JOBS in job_counts:
    PROC = generateData(N_JOBS=N_JOBS)
    print(f"\nComparing for N_JOBS = {N_JOBS} with Genetic Algorithm:")
    print_genetic_schedule(PROC, N_JOBS, N_MACHINES)
    print("\n")
